In [1]:
import os

import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

os.environ['WDM_LOG_LEVEL'] = "false"

import logging
logging.getLogger('WDM').setLevel(logging.NOTSET)

### DATA SOURCE: https://www.tff.org/default.aspx?pageID=523

In [2]:
def get_driver():

    options = webdriver.ChromeOptions()
    # options.headless = True
    options.add_argument('--log-level=3')
    options.add_argument("--silent")
    options.add_argument('--disable-gpu')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = options)

    return driver

In [3]:
def select_city(driver):
    
    # Select Tab
    (
        driver.find_element(by="css selector", value="div[class='levelwrap level1']")
        .find_elements(by="tag name", value="li")[1]
        .find_element(by="tag name", value="a")
        .click()
    )

    # Click Dropdown
    dropdown = driver.find_element(by="css selector", value="div[id*='_SehirSelector1_combo_wrapper']")
    dropdown.click()

    # Select Option
    option = dropdown.find_element(by="xpath", value=f"//div[contains(text(), 'İSTANBUL')]")
    option.click()

    #  # Click Search
    submit_button = driver.find_element(
        by="css selector", 
        value="table[class*='bilgiBankasiArama'] tr:last-child td:last-child input[id*='_btnSearch2']"
    )

    driver.execute_script("arguments[0].click();", submit_button)

    return driver

In [4]:
stad_list = []
url="https://www.tff.org/default.aspx?pageID=523"
driver=get_driver()
driver.get(url)

driver = select_city(driver=driver)

cnt=0
while cnt<2:
    soup = BeautifulSoup(driver.page_source, "lxml")
    rows = soup.select("table.MasterTable_TFF_Contents > tbody")[0].find_all("tr")
    stad_list.extend([row.find_all("td")[0].a.text for row in rows])

    # Click next page
    (
        driver.find_element(by="tag name", value="tfoot")
        .find_elements(by="tag name", value="a")[-1]
        .click()
    )

    cnt+=1

driver.quit()
print('Done!')


Done!


In [5]:
df = pd.DataFrame({"stadium_names" : stad_list})
df.to_csv("tr_ist_stadiumns.csv", index=False)